<a href="https://colab.research.google.com/github/KrishnaNarode/PDF-RAG-System/blob/main/RAG_Proogression_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-core langchain-tavily faiss-cpu pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI Key: ")
os.environ["TAVILY_API_KEY"] = getpass("Enter TAVILY Key: ")


Enter OPENAI Key: ··········
Enter TAVILY Key: ··········


In [ ]:
from google.colab import files
import tempfile
import os
uploaded = files.upload()

filename = list(uploaded.keys())[0]

# get the file path and store in temporary location
file_path = os.path.join(tempfile.gettempdir(), filename)
with open(file_path, "wb") as f:
        f.write(uploaded[filename])

Saving AICTE VOIS Offer Letter 2001-4K-708.pdf to AICTE VOIS Offer Letter 2001-4K-708 (3).pdf


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_tavily import TavilySearch
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
tool = TavilySearch(max_results = 3)
text_splitters = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS

def process_pdf(file):
    loader = PyMuPDFLoader(file)
    docs = loader.load()
    chunks = text_splitters.split_documents(docs)
    vector_store = FAISS.from_documents(chunks, embedding_model)
    return vector_store

In [ ]:
vector_store = process_pdf(filename)

In [ ]:
retriever = vector_store.as_retriever(search_kwargs = {"k": 3})

In [ ]:
from langchain_core.prompts import PromptTemplate

answer_determiniation_prompt = PromptTemplate.from_template("""
You are an AI assistant tasked with determining if the provided context from a PDF contains sufficient information to answer a user's question.

Context from PDF: {context}

User Question: {question}

First, carefully analyze if the context provides adequate information to answer the question.

If the context contains sufficient information to answer the question, respond with a complete and accurate answer based ONLY on the provided context.

If the context does NOT contain sufficient information to fully answer the question, respond with exactly: "[NEED_WEB_SEARCH]"

Your response:
""")

web_search_prompt = PromptTemplate.from_template("""
You are an AI assistant helping a user with their question.

User Question: {question}

Web Search Results: {web_results}

Using the web search results, provide a comprehensive and accurate answer to the user's question.
Make sure to cite sources from the search results where appropriate.
""")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

determination_chain = (
  {
    "context": retriever | format_docs,
    "question": RunnablePassthrough(),
  }
  | answer_determiniation_prompt
  | llm
  | StrOutputParser()
)

web_search_chain = (
  {
    "question": RunnablePassthrough(),
    "web_results": lambda x: tool.invoke({"query": x})
  }
  | web_search_prompt
  | llm
  | StrOutputParser()
)

In [ ]:
def agent(question):
  pdf_response = determination_chain.invoke(question)
  if "[NEED_WEB_SEARCH]" in pdf_response:
    print("\n\u2139\ufe0f Info not found in PDF. Searching the web...")
    return web_search_chain.invoke(question)
  else:
    return pdf_response

In [ ]:
# Ask questions interactively
while True:
  query = input("\nAsk a question about your PDF (or type 'exit'): ")
  if query.lower() == 'exit':
    break
  answer = agent(query)
  print("\n✉️ Answer:", answer)


Ask a question about your PDF (or type 'exit'): Intership time?

✉️ Answer: The internship period is from 10th September 2025 to 8th October 2025.

Ask a question about your PDF (or type 'exit'): exit
